# Performance of the old ("alternative") fingerprint

Make the error plots and the parity plots

In [3]:
import pdb
import copy
from pprint import pprint
import numpy as np
import dill as pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from sklearn.gaussian_process import GaussianProcessRegressor
from tpot import TPOTRegressor
from gaspy.utils import vasp_settings_to_str, print_dict
from gaspy import defaults
from gaspy_regress.regressor import GASpyRegressor

rcParams['font.sans-serif'] = 'Arial'
rcParams['font.family'] = 'sans-serif'
rcParams['savefig.dpi'] = 600

%matplotlib inline
%load_ext ipycache

The ipycache extension is already loaded. To reload it, use:
  %reload_ext ipycache


# Perform regression

In [2]:
# Regression settings
VASP_SETTINGS = vasp_settings_to_str({'gga': 'RP',
                                      'pp_version': '5.4',
                                      'encut': 350})
model_name = 'TPOT'
inner_features = ['coordcount']
outer_features = ['neighbors_coordcounts']
responses = ['energy']
fingerprints = defaults.fingerprints()
blocks = ['adsorbate']
fit_blocks = [('CO',), ('H',)]

# Regression method
tpot = TPOTRegressor(generations=1,
                     population_size=16,
                     offspring_size=16,
                     verbosity=2,
                     scoring='neg_median_absolute_error',
                     n_jobs=16)
gp = GaussianProcessRegressor()

# Perform the regression
model = GASpyRegressor(features=inner_features, responses=responses,
                       fingerprints=fingerprints,
                       blocks=blocks, vasp_settings=VASP_SETTINGS,
                       dim_red='pca', train_size=0.9)
model.fit_tpot(tpot, model_name=model_name, blocks=fit_blocks)
model.fit_hierarchical(gp, 'fit_sk', outer_features,
                       blocks=fit_blocks, dim_red='pca', n_components=0.95)

/project/projectdirs/m2755/GASpy_conda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



Starting to pull documents...


49756it [00:01, 39204.56it/s]
/project/projectdirs/m2755/GASpy_conda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



Generation 1 - Current best internal CV score: 0.166617910201

Best pipeline: KNeighborsRegressor(CombineDFs(input_matrix, input_matrix), n_neighbors=78, p=1, weights=distance)


Generation 1 - Current best internal CV score: 0.147346222269

Best pipeline: GradientBoostingRegressor(input_matrix, alpha=0.95, learning_rate=0.1, loss=huber, max_depth=8, max_features=0.2, min_samples_leaf=12, min_samples_split=3, n_estimators=100, subsample=0.65)


# Calculate metrics

In [5]:
print('RMSE:')
print_dict(model.calc_performance_metric(metric='rmse'), indent=1)
print('MAE:')
print_dict(model.calc_performance_metric(metric='mae'), indent=1)
print('MAD:')
print_dict(model.calc_performance_metric(metric='mad'), indent=1)

RMSE:
	('H',)
		test
			0.925326355268
		all
			0.351057432418
		train
			0.209638957788
	('CO',)
		test
			0.899873546234
		all
			0.321830559719
		train
			0.152765195072
MAE:
	('H',)
		test
			0.236058016552
		all
			0.104787299976
		train
			0.0905867497993
	('CO',)
		test
			0.220672196535
		all
			0.0828115522912
		train
			0.0671479146127
MAD:
	('H',)
		test
			0.126122110083
		all
			0.03554526821
		train
			0.0259410981807
	('CO',)
		test
			0.0912311602337
		all
			0.019393259687
		train
			0.0136967151413
